# Model Selection

Source: https://github.com/gxercavins/time-series/tree/master

The next step is to select a model to fit the data (refer now to the model_selection.ipynb file). For this example I have chosen to use SARIMA, which stands for 'Seasonal AutoRegressive Integrated Moving Averages'. SARIMA is a well-known statistical method for time series regression. Its Python implementation is found in the statsmodels package.

ARIMA models are characterized by three parameters: (p, d, q). These capture the auto-regressive, integrated and moving average components of the model, respectively.

p incorporates the effect of past values into the model. Intuitively, it is likely to be warm tomorrow if it has been warm the past p days. d incorporates the amount of differencing. Intuitively, it is likely to be same temperature tomorrow if the difference in temperature in the last d days has been very small. q sets the error of our model as a linear combination of the error values observed at the previous q time points, thus accounting for the moving average part.

Seasonal ARIMA extends the ARIMA model with a seasonal or periodic component. It adds three parameters (P, Q, D), which are analogous to the previous ones but refer to the seasonal terms. In this case, weekly periodicity was selected (s = 24*7 = 168). Fine-tuning all these parameters can be a difficult task. For that reason, grid search or hyperparameter tuning techniques are usually applied. An implementation is provided in the script, however it is time and resource consuming. If the s parameter is relaxed to 24 (daily periodicity) the execution is much faster and can be done in practical time with a commodity computer. However, results fail to predict differences between workdays and weekend days. 

The metric used to benchmark parameter performance is AIC (Akaike Information Criterion), a common figure of merit for ARIMA models. AIC rewards how well the model fits to the input data but penalizes complexity. The following were the best results obtained:


((p, d, q), (P, D, Q, S), AIC)
------------------------------

((1, 1, 1), (1, 1, 0, 168), 3818.95)

((1, 0, 0), (1, 1, 0, 168), 3835.74)

((1, 0, 1), (1, 1, 0, 168), 3836.72)

((0, 0, 1), (1, 1, 0, 168), 3854.2)

((0, 1, 1), (1, 1, 0, 168), 3877.28)


The best one is selected for use in the final forecast.ipynb file, used to predict the up-coming week results. Week 5 predictions are written to a results.xlsx file and shown below:

In [1]:
import warnings

warnings.filterwarnings("ignore") # specify to ignore warning messages

import pandas as pd
import matplotlib.pyplot as plt
import itertools as it
import datetime
import math

import statsmodels.api as sm

In [2]:
xl = pd.ExcelFile('input_data.xlsx')

data = {sheet_name: xl.parse(sheet_name) for sheet_name in xl.sheet_names}

days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

res = []

for name in xl.sheet_names:
    mylist = map(list, zip(*data[name].values))
    aux = list(it.chain(*mylist))
    del aux[:24]
    res.append(aux)

res = list(it.chain(*res))

days = [days for i in range(0, 4)]
days_repeated = days * 4

In [4]:
# Tune Seasonal ARIMA model
# Define the p, d and q parameters to take any value between 0 and 2
p = d = q = range(0,2)

# Generate all different combinations of p, d and q triplets
pdq = list(it.product(p, d, q))
print(pdq)

# Generate all different combinations of seasonal p, d and q triplets
# Seasonality is one week (24*7 = 168 hours)
seasonal_pdq = [(x[0], x[1], x[2], 168) for x in list(it.product(p, d, q))]

print('Examples of parameter combinations for Seasonal ARIMA...')
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))

[(0, 0, 0), (0, 0, 1), (0, 1, 0), (0, 1, 1), (1, 0, 0), (1, 0, 1), (1, 1, 0), (1, 1, 1)]
Examples of parameter combinations for Seasonal ARIMA...
SARIMAX: (0, 0, 1) x (0, 0, 1, 168)
SARIMAX: (0, 0, 1) x (0, 1, 0, 168)
SARIMAX: (0, 1, 0) x (0, 1, 1, 168)
SARIMAX: (0, 1, 0) x (1, 0, 0, 168)


In [4]:
result_list = []

for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(res,
                                            order=param,
                                            seasonal_order=param_seasonal,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)
            results = mod.fit()

            print('ARIMA{}x{}168 - AIC:{}'.format(param, param_seasonal, round(results.aic,2)))
            result_list.extend([param, param_seasonal, round(results.aic,2)])
        except:
            print('error')
            continue
            
print('Done!')

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            1     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  7.51148D+00    |proj g|=  3.25731D-06

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    1      0      1      0     0     0   3.257D-06   7.511D+00
  F =   7.5114827111438993     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            
ARIMA(0, 0, 0)x(0, 0, 0, 168)168 - AIC:10097.43


 This problem is unconstrained.
 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            2     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.81467D+01    |proj g|=  1.47213D-03
  ys=-5.542E+01  -gs= 6.983E-01 BFGS update SKIPPED


In [30]:
print_result = zip(*[iter(result_list)]*3) 
print_result.sort(key=lambda x: x[2])

print('Result summary:\n')
print('((p, d, q), (P, D, Q, S), AIC)')
print('------------------------------')
for item in print_result:
    print(item)

Result summary:

((p, d, q), (P, D, Q, S), AIC)
------------------------------
((1, 1, 1), (1, 1, 0, 168), 3818.95)
((1, 0, 0), (1, 1, 0, 168), 3835.74)
((1, 0, 1), (1, 1, 0, 168), 3836.72)
((0, 0, 1), (1, 1, 0, 168), 3854.2)
((0, 1, 1), (1, 1, 0, 168), 3877.28)
((1, 1, 0), (1, 1, 0, 168), 3912.87)
((0, 0, 0), (1, 1, 0, 168), 3915.73)
((0, 1, 0), (1, 1, 0, 168), 3939.41)
((1, 1, 1), (0, 1, 0, 168), 5934.29)
((1, 0, 1), (0, 1, 0, 168), 5946.06)
((1, 1, 1), (1, 0, 0, 168), 5948.39)
((0, 0, 1), (0, 1, 0, 168), 5953.08)
((1, 0, 0), (0, 1, 0, 168), 5956.36)
((1, 0, 0), (1, 0, 0, 168), 5956.75)
((1, 0, 1), (1, 0, 0, 168), 5957.24)
((0, 0, 1), (1, 0, 0, 168), 5973.03)
((0, 1, 1), (0, 1, 0, 168), 6007.22)
((0, 1, 1), (1, 0, 0, 168), 6028.44)
((0, 0, 0), (1, 0, 0, 168), 6050.84)
((0, 0, 0), (0, 1, 0, 168), 6053.09)
((1, 1, 0), (0, 1, 0, 168), 6084.03)
((1, 1, 0), (1, 0, 0, 168), 6085.09)
((0, 1, 0), (0, 1, 0, 168), 6114.69)
((0, 1, 0), (1, 0, 0, 168), 6121.6)
((1, 0, 1), (0, 0, 0, 168), 8866.63